# **Сети Колмогорова-Арнольда (KAN)**
---
## **1. Историческое развитие**

Теорему о представлении непрерывных функций нескольких переменных через суперпозиции непрерывных функций одной переменной и сложения, вставшую в основу KAN, сформулировал Андрей Колмогоров в 1957 году. В 1963 году Владимир Арнольд уточнил представление Колмогорова, показав, как именно можно построить функции от одной переменной, тем самым Арнольд придал теореме Колмогорова практическую форму. Именно этот результат стал известен как теорема Колмогорова-Арнольда.

Эта теорема, сформулированная в середине XX века, долгое время считалась теоретически значимой, но практически бесполезной из-за сложности реализации и негладкости внутренних функций, значительно усложняющих обучение.

---

## **2. Математические основы**

### **2.1 Теорема Колмогорова-Арнольда**

Из теоремы Колмогорова-Арнольда следует, что если $f$ — многомерная непрерывная функция в ограниченной области, то $f$ можно записать как конечную композицию непрерывных функций одной переменной и бинарной операции сложения следующим образом: $$f(x_1, \dots, x_n) = \sum_{q=1}^{2n+1} \Phi_q \left( \sum_{p=1}^{n} \phi_{q,p}(x_p) \right)
$$

### **2.2 Проблемы теоремы Колмогорова-Арнольда**

Теорему до недавнего времени не рассматривали в качестве основы для создания нейронных сетей по двум причинам:

1. **Негладкость одномерных функций.**
Теорема предполагает разложение многомерной функции на суперпозицию одномерных, но эти одномерные функции могут быть негладкими. Это делает невозможным применение метода обратного распространения ошибки (backpropagation), который требует дифференцируемости функций.

2. **Жёсткая фиксация структуры.**
В оригинальной теореме архитектура сети строго ограничена: всего два слоя и $2n + 1$ нейронов в скрытом слое (где $n$ — размерность входных данных). Такая структура слишком проста для большинства практических задач и не обладает гибкостью, необходимой для адаптации к сложным данным.

### **2.3 Решение проблем теоремы Колмогорова-Арнольда в KAN**

1. **B-сплайны.** Вместо произвольных функций KAN используют гладкие B-сплайны для представления активационных функций. Это гарантирует их дифференцируемость и позволяет применять современные методы обучения. Каждая активационная функция имеет вид:

$$
\phi(x) = w_b \cdot b(x) + w_s \cdot \mathrm{spline}(x)
$$,

$b(x)$ - базовая гладкая функция, $\mathrm{spline}(x)$ - линейная комбинация B-сплайнов:

$$
\mathrm{spline}(x) = \sum_{i} c_i B_i(x)
$$

2. **Глубокая композиция.** KAN вводят понятие слоя как матрицы функций:

$$
Φ_l = \{\phi_{l,j,i}\}, \quad i = 1..n_{in}, \quad j = 1.n_{out}
$$

где $\phi_{l,j,i}$ - обучаемая 1D функция параметризованная сплайном.

Выход KAN вычисляется как композиция L таких слоев:

$$
\text{KAN}(\mathbf{x}) = \left( \Phi_{L-1} \circ \Phi_{L-2} \circ \cdots \circ \Phi_0 \right) \mathbf{x}
$$

Что также можно нагляднее представить как:

$$
f(\mathbf{x}) = \sum_{i_{L-1}} \phi_{L-1, i_L, i_{L-1}} \left(
    \sum_{i_{L-2}} \cdots \left(
        \sum_{i_1} \phi_{1, i_2, i_1} \left(
            \sum_{i_0} \phi_{0, i_1, i_0}(x_{i_0})
        \right)
    \right) \cdots
\right)
$$

---

## **3. Основная структура**


### **3.1 Представление слоев**
Архитектура задается списком:

$$
[n_0, n_1, \dots, n_L]
$$

где $n_l$ - количество нейронов в слое $l$.

### **3.2 Функция активации**
Каждая связь между нейронами $(l, i)$ и $(l + 1, j)$ имеет свою функцию активации:

$$
\phi_{l,j,i}: ℝ → ℝ
$$

Параметризованную как:

$$
\phi_{l,j,i}(x) = w_b \cdot b(x) + w_s \cdot \mathrm{spline}(x)
$$

### **3.3 Вычисления в слое**

Выход нейрона (в данном случае нейрона $(l +1, j)$) вычисляется как сумма входящих в него функций:

$$
x_{l+1, j} = \sum_{i=1}^{n_l}\phi_{l,j,i}(x_{l, i})
$$

Если выражать это в матричной форме:

$$
x_{l+1} = Φ_lx_l
$$

где - функциональная матрица:

$$
\Phi_l =
\begin{bmatrix}
\phi_{l,1,1} & \cdots & \phi_{l,1,n_l} \\
\vdots & \ddots & \vdots \\
\phi_{l,n_{l+1},1} & \cdots & \phi_{l,n_{l+1},n_l}
\end{bmatrix}
$$

### **3.4 Полная сеть**

Полная сеть уже была описана выше и выглядит следующим образом:

$$
\text{KAN}(\mathbf{x}) = \left( \Phi_{L-1} \circ \Phi_{L-2} \circ \cdots \circ \Phi_0 \right) \mathbf{x}
$$

Или в другой форме:

$$
f(\mathbf{x}) = \sum_{i_{L-1}} \phi_{L-1, i_L, i_{L-1}} \left(
    \sum_{i_{L-2}} \cdots \left(
        \sum_{i_1} \phi_{1, i_2, i_1} \left(
            \sum_{i_0} \phi_{0, i_1, i_0}(x_{i_0})
        \right)
    \right) \cdots
\right)
$$

---

## 4. **Сравнение KAN с MLP**

Многослойные перцептроны долгое время остаются стандартом для аппроксимации нелинейных функций в глубоком обучении. Благодаря универсальной теореме аппроксимации и простоте реализации, MLP легли в основу практически всех современных нейросетевых архитектур.

### 4.1 **Функции активации**
- **MLP**: Нелинейные функции активации (ReLU, tanh, сигмоида и т.д.) одинаковые для всего слоя, расположены на узлах.
- **KAN**: 	Обучаемые нелинейные B-сплайны разные для каждого входа, расположены на ребрах.

### 4.2 **Параметризация**
- **MLP**:
  - Линейные веса: $W \in \mathbb{R}^{n_{l+1} \times n_l}$
  - Фиксированные активации: $\sigma(Wx + b)$
- **KAN**:
  - Обучаемые функции: $\phi_{l,j,i}(x) = w_b b(x) + w_s \text{spline}(x)$
  - Нет линейных весов - только параметры сплайнов

### 4.3 **Стратегия аппроксимации**

* **MLP**: Глобальная аппроксимация — каждый нейрон участвует в формировании всего выходного пространства, обучение распределяет информацию по всей сети.
* **KAN**: Локальная аппроксимация — B-сплайны действуют в ограниченных интервалах, что позволяет точно аппроксимировать функции с локальными особенностями.

### 4.4 **Метод обучения**

* **MLP**: Сквозное обучение (end-to-end) методом обратного распространения ошибки (backpropagation).
* **KAN**: Также используется backpropagation, благодаря дифференцируемости сплайнов.

### 4.5 **Скорость обучения**

* **MLP**: Обычно быстрее обучается благодаря простоте операций и хорошо проработанным оптимизаторам.
* **KAN**: Медленнее из-за более сложной архитектуры и численно затратных операций со сплайнами.

### 4.6 **Размер и вычислительная эффективность**

* **MLP**: Требует больше параметров, но оптимален по числу FLOPs (Floating Point Operations).
* **KAN**: Для достижения сопоставимой точности требует меньше параметров, но больше вычислений из-за стоимости сплайнов (больше FLOPs на одно соединение).

### 4.7 **Интерпретируемость**

* **MLP**: Низкая — веса и активации сложно интерпретировать, особенно в глубоких сетях.
* **KAN**: Выше — каждая функция на ребре явно задана (например, через B-сплайн), что позволяет анализировать вклад каждого входа в выход.

---

## **5. Методология обучения KAN**

Обучение KAN строится на тех же принципах, что и обучение классических нейросетей, но с рядом особенностей, связанных с обучаемыми функциями активации и нестандартной структурой параметров.

### **5.1 Инициализация параметров**

Обучение начинается с случайной инициализации параметров сети. В отличие от MLP, где инициализируются веса и смещения, в KAN инициализируются:

* коэффициенты B-сплайнов,
* весовые коэффициенты линейной составляющей (если используется),

### **5.2 Прямой и обратный проход**

Процесс обучения включает два этапа:

1. **Прямой проход (forward pass)**: входные данные проходят через сеть, на каждом ребре применяются индивидуальные B-сплайны, результат агрегируется и формирует выход модели.
2. **Обратный проход (backward pass)**: вычисляется ошибка между предсказанием и истинной меткой, после чего с помощью правила цепочки (chain rule) дифференцирования вычисляются градиенты по всем параметрам.

Параметры обновляются с помощью стандартных методов оптимизации, таких как градиентный спуск, стохастический градиентный спуск (SGD), Adam и т.д.

### **5.3 Проблемы стабильности и регуляризация**

Одной из ключевых сложностей при обучении KAN является обеспечение стабильности и сходимости процесса оптимизации. Это связано с тем, что обучаемые функции активации (B-сплайны) имеют сложную зависимость от параметров и могут вызывать переобучение или нестабильные градиенты.

Для решения этих проблем применяются похожие с MLP методы:

* **Dropout** — для предотвращения переобучения;
* **Weight decay** — для ограничения роста параметров;
* **Batch normalization** и **Layer normalization** — для стабилизации распределений активаций и ускорения сходимости.

### **5.4 Сквозное обучение**

KAN поддерживают сквозное (end-to-end) обучение, при котором все параметры сети обучаются одновременно. Поддержка backpropagation и end-to-end обучения позволяет интегрировать KAN в современные фреймворки глубокого обучения (например, PyTorch), использовать его вместе с другими архитектурными блоками (CNN, Transformer) и обучать модель без ручной настройки отдельных этапов.

---

## **6. Практические применения**

KAN демонстрируют высокую точность и эффективность в ряде задач, где требуется точная аппроксимация, сохранение информации и компактность модели.
Далее будут представлены результаты экспериментов из статьи KAN: Kolmogorov–Arnold Networks (https://arxiv.org/pdf/2404.19756).

### **6.1 Аппроксимация аналитических и символьных функций**

KAN превосходно подходят для символьной регрессии — задачи восстановления математической формулы по точкам. В экспериментах:

* KAN с тем же количеством параметров, что и MLP, достигали меньших ошибок на обучении и тесте.
* Особенно заметно преимущество при увеличении размерности входов и усложнении функции: глубокие KAN масштабируются лучше, чем глубокие MLP.

### **6.2 Приближение специальных функций**

KAN эффективно аппроксимируют специальные математические функции, широко используемые в физике и инженерии (например, функции Бесселя, Лежандра и сферические гармоники).

* В тесте на 15 таких функций KAN показали устойчиво меньшую ошибку на обучении и тесте, чем MLP, при одинаковом числе параметров.
* Компактные структуры KAN обеспечивали точность, сопоставимую с MLP, в которых было в десятки и сотни раз больше параметров.

### **6.3 Решение дифференциальных уравнений (PDE)**

KAN демонстрируют впечатляющую эффективность в задачах численного решения уравнений в частных производных:

* KAN с 2 слоями шириной 10 показали точность $10^{-7}$,
* В то время как MLP с 4 слоями шириной 100 достигали только $10^{-5}$,
* При этом KAN использовал в 100 раз меньше параметров.

### **6.4 Continual Learning**

KAN устойчивы к проблеме катастрофического забывания, благодаря своей локальной природе B-сплайнов:

* При обучении новому примеру изменяются только локальные участки B-сплайнов.
* Остальные части сети остаются нетронутыми — сеть сохраняет ранее выученные знания.
* В отличие от MLP, где обновления могут влиять на весь выход, в KAN правки минимальны и изолированы.

Это делает их подходящими для задач, где данные приходят поэтапно, как в обучении на потоке данных или в реальных временных системах.

### **6.5 Компактные модели для сложных задач**

Как уже было представлено в экспериментах выше, KAN больше всего выигрывает в требовательности к числу параметров:

* KAN с 200 параметрами достиг 81.6% точности в задаче предсказания геометрического инварианта узлов (knot classification),
* В то время как модель от Google DeepMind (MLP с \~300,000 параметров) — только 78% точности.

Это подчёркивает высокую параметрическую эффективность KAN.

---

## **7. Сильные и слабые стороны**

### **7.1 Сильные стороны**

- **Высокая выразительная способность**

KAN эффективно аппроксимируют сложные математические зависимости и специальные функции даже при малом числе параметров. Их обучаемые B-сплайны позволяют точно передавать локальные особенности функций.

- **Параметрическая эффективность**

Для достижения той же точности, что и MLP, KAN требует в десятки или сотни раз меньше параметров.

- **Интерпретируемость**

Каждая связь в KAN представлена явной функцией, что позволяет анализировать, какой именно вход как влияет на выход, в отличие от "черного ящика" в MLP.

- **Обучение без забывания (continual learning)**

Локальная природа сплайнов позволяет KAN сохранять ранее выученные знания, не разрушая их при обучении на новых данных. Это делает модель устойчивой к катастрофическому забыванию.

### **7.2 Слабые стороны**

- **Сложность обучения**

Обучение KAN требует аккуратной настройки: неправильная инициализация, слишком высокий learning rate или плохая структура сплайна могут привести к нестабильности или затуханию градиентов.

- **Более высокая вычислительная стоимость (FLOPs)**

Хотя число параметров у KAN меньше, число операций (FLOPs) больше, чем у MLP. Это делает KAN медленнее в задачах, где важна скорость, особенно на GPU.

- **Недостаток фреймворков и библиотек**

KAN пока не так широко распространены, как MLP. Поддержка во фреймворках ограничена, а автоматический подбор архитектур и оптимизация только развивается.

- **Недостаток практических исследований**

По сетям Колмогорова-Арнольда на данный момент мало исследований, при этом некоторые из них противоречат друг другу в результатах экспериментов. К примеру, в оригинальной статье [1] в плюс KAN ставится continual learning и устойчивость к забыванию, но в более поздней статье [2] указывается на то, что в таких задачах как, например, CV, KAN в contunual learning наоборот проявляет себя хуже, чем MLP.

---

## **8. Перспективы развития KAN**

Сети Колмогорова-Арнольда стали активно развивать в 2024 году, после публикации статьи [1], в которой были преодолены основные проблемы из пункта 2.2. В данный момент исследований на эту тему недостаточно, но уже можно выделить основные векторы развития и проблемы, которые нужно будет решить.

### **8.1 Расширение прикладных областей**

Хотя KAN показали отличные результаты в задачах аппроксимации аналитических функций, символьной регрессии и решения уравнений в частных производных, дальнейшее развитие требует их адаптации к более прикладным областям, включая компьютерное зрение, обработку естественного языка и табличные данные. Уже начались попытки встраивания KAN в визуальные архитектуры (например, Vision-KAN [3] и U-KAN [4]).

### **8.2 Совершенствование обучения**

Одним из препятствий для широкого применения KAN остаются сложности в обучении: высокая вычислительная стоимость и нестабильность градиентов. Перспективным направлением является развитие более устойчивых оптимизационных подходов, включая гибридные схемы (например, чередование Adam и LBFGS, предложенное в [1]) и автоматическую настройку структуры сплайнов.

Также может быть полезным использование техник из области нейроэволюции или нейроархитектурного поиска (NAS) для автоматического подбора формы KAN и оптимального распределения контрольных точек.

### **8.3 Интерпретируемость и научные применения**

KAN демонстрируют потенциал как инструмент объяснимого машинного обучения (XAI). Возможность анализировать каждую обученную функцию на ребре делает KAN привлекательными для приложений в науке и инженерии, где важно понимать, как модель пришла к выводу. Исследования компактных представлений специальных функций открывают возможности для использования KAN как инструмента автоматического поиска символических выражений или даже новых математических закономерностей.

---

## Заключение

Сети Колмогорова-Арнольда представляют собой перспективную архитектуру, сочетающую в себе гибкость, интерпретируемость и высокую точность аппроксимации. Использование обучаемых B-сплайнов на уровне соединений позволяет KAN уже превосходить классические MLP в некоторых задачах. Однако сети Колмогорова-Арнольда имеют много весомых недостатков, более того, несмотря на успехи первых работ, теоретическая база и практический опыт пока очень ограничены — многие заявленные преимущества нуждаются в подтверждении в более широком спектре прикладных задач и при использовании современных методов обучения. Тем не менее, потенциал KAN очевиден.

## **Источники**

[1] Liu, Z. (2024). Kolmogorov–Arnold Networks (KANs). arXiv:2404.19756

[2] Runpeng Yu, Weihao Yu, and Xinchao Wang (2024). KAN or MLP: A Fairer Comparison. arXiv:2407.16674

[3] Chenziwen Haoshuai. (2024). Vision-KAN. GitHub repository. https://github.com/chenziwenhaoshuai/Vision-KAN

[4] Gu, W., Wang, X., & Yu, R. (2024). U-KAN makes strong backbone for medical image segmentation and generation. arXiv preprint, arXiv:2406.02918